In [1]:
#hide
from fastbook import *
from IPython.display import display,HTML

/home/gg/.local/lib/python3.9/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


In [2]:
from fastai.text.all import *

In [3]:
class NormalizeTr(Transform):
    def setup(self, items):
        pass
    def encodes():
        pass
    def decodes():
        pass

In [4]:
NormalizeTr?

Init signature: NormalizeTr(self, enc=None, dec=None, split_idx=None, order=None)
Docstring:      Delegates (`__call__`,`decode`,`setup`) to (<code>encodes</code>,<code>decodes</code>,<code>setups</code>) if `split_idx` matches
File:           ~/.local/lib/python3.9/site-packages/fastcore/transform.py
Type:           _TfmMeta
Subclasses:     


In [5]:
Transform??

Init signature: Transform(self, enc=None, dec=None, split_idx=None, order=None)
Source:        
class Transform(metaclass=_TfmMeta):
    "Delegates (`__call__`,`decode`,`setup`) to (<code>encodes</code>,<code>decodes</code>,<code>setups</code>) if `split_idx` matches"
    split_idx,init_enc,order,train_setup = None,None,0,None
    def __init__(self, enc=None, dec=None, split_idx=None, order=None):
        self.split_idx = ifnone(split_idx, self.split_idx)
        if order is not None: self.order=order
        self.init_enc = enc or dec
        if not self.init_enc: return

        self.encodes,self.decodes,self.setups = TypeDispatch(),TypeDispatch(),TypeDispatch()
        if enc:
            self.encodes.add(enc)
            self.order = getattr(enc,'order',self.order)
            if len(type_hints(enc)) > 0: self.input_types = first(type_hints(enc).values())
            self._name = _get_name(enc)
        if dec: self.decodes.add(dec)

    @property
    def name(self): return getattr(

In [6]:
@Transform
def get_mean(x):
    mean = sum(x)/float(len(x))
    return mean

@Transform
def get_std(x):
    mean = sum(x)/float(len(x))
    std = (sum([abs(i-mean)**2 for i in x])/float(len(x)))**0.5
    return std

class NormalizeTr(Transform):
    def setup(self, items):
        self.items = items
        self.mean = get_mean(self.items)
        self.std = get_std(self.items)
    def encodes(self):
        return (self.items - self.mean)/self.std
    def decodes(self):
        return self.items

In [7]:
tfm = NormalizeTr()

In [8]:
x = np.arange(0,100)

In [9]:
x.mean(), x.std()

(49.5, 28.86607004772212)

In [10]:
tfm.setup(x)

In [11]:
tfm.mean, tfm.std

(49.5, 28.86607004772212)

In [12]:
tfm.decodes()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47,
       48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95,
       96, 97, 98, 99])

# Use transcripts to tokenize

In [90]:
path = Path('/home/gg/knowKenny_through_chaiTime/subtitle_formatted/')

In [91]:
file = path/'subtitle_KennySebastian_MIDDLECLASSRESTAURANTPROBLEMS STANDUPCOMEDY-KennySebastian.txt'

In [92]:
text = file.open().read()

In [140]:
class NumericalizeTr(Transform):
    def setups(self, items): 
        self.items_copy = items
        self.unique_words = L([])  # vocab
    def encodes(self, items): 
        self.counts, self.items_num = self.NumberWords(items)
        return self.items_num
    def decodes(self,items_nums):  # can be any item nums
        decoded = L([L(self.counts.keys())[w_id] for w_id in items_nums])
        return decoded
    def NumberWords(self,items): 
        new_words = [w for w in items.unique() if w not in self.unique_words]
        [self.unique_words.append(i) for i in new_words] # update the vocab
        self.counts = {word : [idx, sum([True for w in items if w == word])] for idx,word in enumerate(self.unique_words)}
        # counts only for new list
        # counts.keys, counts.values()
        # assign the ids to each word in tokens
        # make a dict with a list as value 0 is idx and 1 is the count 
        self.x_num = L([self.counts.get(w)[0] for w in items]) # get ids of each word
        return self.counts, self.x_num

In [121]:
w_tok = WordTokenizer()

In [122]:
tok = Tokenizer(w_tok)

In [123]:
tokens = tok(text)

In [124]:
tokens

(#1475) ['xxbos','i','just','got','back','from','xxmaj','atlanta','it','was'...]

In [125]:
num = NumericalizeTr()

In [126]:
num.setup(tokens)

In [127]:
num(tokens)

(#1475) [0,1,2,3,4,5,6,7,8,9...]

In [128]:
num.decodes([4,5,10])

(#3) ['back','from','a']

In [129]:
num.unique_words

(#440) ['xxbos','i','just','got','back','from','xxmaj','atlanta','it','was'...]

## test with new tokens

In [130]:
file = path/'subtitle_TheMelbComedyFest_KaruneshTalwar-ComedyUpLate2018(S6,E10).txt'
text = file.open().read()

In [131]:
tokens_new = tok(text)

In [132]:
tokens_new

(#834) ['xxbos','xxmaj','applause','we','here','at','the','xxmaj','melbourne','xxmaj'...]

In [133]:
# num.setup(tokens_new)# dont call setup again, else will init new empty vocab list

In [134]:
num(tokens_new)

(#834) [0,6,440,141,82,177,14,6,441,6...]

In [135]:
num.unique_words

(#618) ['xxbos','i','just','got','back','from','xxmaj','atlanta','it','was'...]

Try one more file

In [136]:
file = path/'subtitle_TrevorNoah_ ItMakesNoSense! -TrevorNoah-(AfricanAmerican).txt'
text = file.open().read()
tokens_new = tok(text)
num(tokens_new)
num.unique_words

(#618) ['xxbos','i','just','got','back','from','xxmaj','atlanta','it','was'...]

In [139]:
#num.counts # counts wont be kept in memory

# Trials

In [ ]:
tokens.unique()

In [ ]:
x = tokens

In [ ]:
unique_words = x.unique()

In [ ]:
counts = {word : [idx,sum([ True for w in x if w == word])] for idx,word in enumerate(unique_words)}
# stores the id and count of each word

In [ ]:
counts.get('xxbos')

In [ ]:
counts.keys()

In [ ]:
L(counts.keys())[0]

In [ ]:
counts.get('xxbos')

In [ ]:
x_num = [counts.get(w)[0] for w in x]
x_num[:10]

In [ ]:
reverse = [L(counts.keys())[w_id] for w_id in x_num]
reverse[:10]

In [ ]:
a = [1,2,3]
b=[3,4,5]
[a.append(i) for i in b]
a